In [ ]:
import requests
import zipfile
import io
import pandas as pd
import mysql.connector

# Define your database connection parameters
db_config = {
  'user': 'your_username',
  'password': 'your_password',
  'host': 'your_host',
  'database': 'temps',
  'raise_on_warnings': True
}

# Establish a connection to the database
cnx = mysql.connector.connect(**db_config)
cursor = cnx.cursor()

# Define the API url
url = 'http://api.climately.com/v1/YOUR_API_KEY/weather/hourly/all.zip'

# Send a HTTP request to the URL
response = requests.get(url)

# Make sure the request was successful
assert response.status_code == 200

# Create a BytesIO object from the response content
zip_file = io.BytesIO(response.content)

# Create a ZipFile object from the BytesIO object
with zipfile.ZipFile(zip_file) as z:
    # Extract all files in the zip file
    z.extractall()

# Load the CSV file into a pandas DataFrame
df = pd.read_csv('data.csv')

# Calculate the wet-bulb temperature
def calculate_wet_bulb_temperature(row):
    # Replace this function with the actual calculation
    return row['temperature']  # Placeholder

df['wet_bulb_temperature'] = df.apply(calculate_wet_bulb_temperature, axis=1)

# Sort the DataFrame by the wet-bulb temperature and get the top 100
df_sorted = df.sort_values('wet_bulb_temperature', ascending=False)
df_top100 = df_sorted.head(100)

# Define the SQL query to insert the data into the database
query = ("INSERT INTO top100 "
         "(station_id, date, wet_bulb_temperature) "
         "VALUES (%s, %s, %s)")

# Insert the top 100 rows into the database
for index, row in df_top100.iterrows():
    data = (row['station_id'], row['date'], row['wet_bulb_temperature'])
    cursor.execute(query, data)

# Commit the changes and close the connection
cnx.commit()
cursor.close()
cnx.close()
